#### Importing the features required for prediction of patent class

In [248]:
import pandas as pd
features = pd.read_csv("./data/patent_class_features_after_2022_20230611.csv")
class_labels = pd.read_csv("./data/patent_class_after_2022_20230611.csv")

In [249]:
features.head()

,PATENT,PUB_DATE,COUNTRY,PRIMARY_LANG,IPC,DWPI_TITLE,DWPI_DETAILED_DESCRIPTIONS,DWPI_NOVELTY,DWPI_USE
0,CN218280954U_,2023-01-13,CN,ZH_ZH,"['B01D004604', 'B01D004676', 'B01D004602', 'B0...",Surface anti-sticking dust-removing filter bag...,NaN,The utility model claims a surface anti-sticki...,NaN
1,CN218188237U_,2023-01-03,CN,ZH_ZH,"['B01D002996', 'B01D002933']",Environment-friendly reusable cooling liquid f...,NaN,The utility model claims a new environment-fri...,NaN
2,CN218442764U_,2023-02-03,CN,ZH_ZH,"['F24H000118', 'B08B000304', 'F24H000902', 'F2...",Organic silicon modified epoxy heat conducting...,NaN,The utility model claims an organic silicon mo...,NaN
3,CN218428090U_,2023-02-03,CN,ZH_ZH,['B25B002700'],"Electronic screen dismounting device, has lowe...",An INDEPENDENT CLAIM is included for the elect...,The utility model relates to screen disassembl...,NaN
4,CN218195410U_,2023-01-03,CN,ZH_ZH,"['B26D000702', 'B26D000510', 'B26D000108']","Inclined cutting paper cutting device, has cut...",NaN,The utility model claims an inclined cutting p...,NaN


In [250]:
class_labels.head()

,PATENT,CLASS
0,CN218280954U_,A88
1,CN218188237U_,A88
2,CN218442764U_,A88
3,CN218428090U_,A88
4,CN218195410U_,A88


In [251]:
unique_class_codes = sorted(class_labels.CLASS.unique())
# Create a dictionary with keys as first letters and values as lists of codes
codes_dict = {}
for code in unique_class_codes:
    first_letter = code[0]
    if first_letter not in codes_dict:
        codes_dict[first_letter] = []
    codes_dict[first_letter].append(code)

# Display the lists for each letter
for letter, codes in codes_dict.items():
    print(f"{letter}: {codes}")

A: ['A11', 'A12', 'A13', 'A14', 'A17', 'A18', 'A21', 'A23', 'A25', 'A26', 'A28', 'A31', 'A32', 'A35', 'A41', 'A60', 'A81', 'A82', 'A83', 'A84', 'A85', 'A86', 'A87', 'A88', 'A89', 'A91', 'A92', 'A93', 'A94', 'A95', 'A96', 'A97']
B: ['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07']
C: ['C01', 'C02', 'C03', 'C04', 'C05', 'C06', 'C07']
D: ['D11', 'D12', 'D13', 'D14', 'D15', 'D16', 'D17', 'D18', 'D21', 'D22', 'D23', 'D24', 'D25']
E: ['E11', 'E12', 'E13', 'E14', 'E15', 'E16', 'E17', 'E19', 'E21', 'E22', 'E23', 'E24', 'E31', 'E32', 'E33', 'E34', 'E35', 'E36', 'E37']
F: ['F01', 'F02', 'F03', 'F04', 'F05', 'F06', 'F07', 'F08', 'F09']
G: ['G01', 'G02', 'G03', 'G04', 'G05', 'G06', 'G07', 'G08']
H: ['H01', 'H02', 'H03', 'H04', 'H05', 'H06', 'H07', 'H08', 'H09']
J: ['J01', 'J02', 'J03', 'J04', 'J05', 'J06', 'J07', 'J08', 'J09']
K: ['K01', 'K02', 'K03', 'K04', 'K05', 'K06', 'K07', 'K08']
L: ['L01', 'L02', 'L03']
M: ['M11', 'M12', 'M13', 'M14', 'M21', 'M22', 'M23', 'M24', 'M25', 'M26', 'M27', 'M28', 

There are only IPC classes and sections under those classes. There are no subclasses and groups mentioned in class codes

In [252]:
# How many IPC sections are there ?
total_keys = len(codes_dict.keys())
print(total_keys)

20


This means I have to create 21 different classifiers. 1 for predicting the starting Alphabet/Class i.e., the root node of the tree. The top level

And then 1 classifier for each Alphabet/Class (20). So, total 21 different classifiers

In [253]:
# How many unique classes are there in each IPC section ?
for letter, codes in codes_dict.items():
    print(f"{letter}: {len(codes)}")

A: 32
B: 7
C: 7
D: 13
E: 19
F: 9
G: 8
H: 9
J: 9
K: 8
L: 3
M: 13
P: 46
Q: 54
S: 6
T: 7
U: 9
V: 8
W: 7
X: 13


In [254]:
features.shape

(344328, 9)

In [255]:
features.columns.dtype

dtype('O')

In [256]:
class_labels.shape

(461258, 2)

In [257]:
features.head()

,PATENT,PUB_DATE,COUNTRY,PRIMARY_LANG,IPC,DWPI_TITLE,DWPI_DETAILED_DESCRIPTIONS,DWPI_NOVELTY,DWPI_USE
0,CN218280954U_,2023-01-13,CN,ZH_ZH,"['B01D004604', 'B01D004676', 'B01D004602', 'B0...",Surface anti-sticking dust-removing filter bag...,NaN,The utility model claims a surface anti-sticki...,NaN
1,CN218188237U_,2023-01-03,CN,ZH_ZH,"['B01D002996', 'B01D002933']",Environment-friendly reusable cooling liquid f...,NaN,The utility model claims a new environment-fri...,NaN
2,CN218442764U_,2023-02-03,CN,ZH_ZH,"['F24H000118', 'B08B000304', 'F24H000902', 'F2...",Organic silicon modified epoxy heat conducting...,NaN,The utility model claims an organic silicon mo...,NaN
3,CN218428090U_,2023-02-03,CN,ZH_ZH,['B25B002700'],"Electronic screen dismounting device, has lowe...",An INDEPENDENT CLAIM is included for the elect...,The utility model relates to screen disassembl...,NaN
4,CN218195410U_,2023-01-03,CN,ZH_ZH,"['B26D000702', 'B26D000510', 'B26D000108']","Inclined cutting paper cutting device, has cut...",NaN,The utility model claims an inclined cutting p...,NaN


In [258]:
# PATENT, DWPI_TITLE and DWPI_NOVELTY are object data type, have to convert them to string
features['PATENT'] = features['PATENT'].astype(str).str.strip()
class_labels['PATENT'] = class_labels['PATENT'].astype(str).str.strip()
features['DWPI_TITLE'] = features['DWPI_TITLE'].astype(str).str.strip()
features['DWPI_NOVELTY'] = features['DWPI_NOVELTY'].astype(str).str.strip()

# Only first 2 characters of PRIMARY_LANG before delimeter "_" is required
def get_left_part(s):
    return s.split('_')[0]
features.PRIMARY_LANG = features.PRIMARY_LANG.apply(get_left_part)

# PUB_DATE column has to be parsed as pandas datetime object
features['PUB_DATE'] = pd.to_datetime(features['PUB_DATE'], format='%Y-%m-%d')
# Calculate the date range
start_date = features['PUB_DATE'].min()
end_date = features['PUB_DATE'].max()
# Print the date range
print(f"Date range: {start_date.date()} to {end_date.date()}")

Date range: 2023-01-01 to 2023-02-28


In [259]:
features.nunique()

PATENT                        344328
PUB_DATE                          59
COUNTRY                           40
PRIMARY_LANG                      23
IPC                           264953
DWPI_TITLE                    343353
DWPI_DETAILED_DESCRIPTIONS     85860
DWPI_NOVELTY                  343944
DWPI_USE                      147691
dtype: int64

In [260]:
missing_values_count = features.isna().sum()
print(missing_values_count)

PATENT                             0
PUB_DATE                           0
COUNTRY                            0
PRIMARY_LANG                       0
IPC                               15
DWPI_TITLE                         0
DWPI_DETAILED_DESCRIPTIONS    251364
DWPI_NOVELTY                       0
DWPI_USE                      193017
dtype: int64


In [261]:
# representing missing values as percentages (proportion)
# note IPC column still has 15 missing values
missing_values_count = round((features.isna().sum()/features.shape[0])*100,4)
print(missing_values_count)

PATENT                         0.0000
PUB_DATE                       0.0000
COUNTRY                        0.0000
PRIMARY_LANG                   0.0000
IPC                            0.0044
DWPI_TITLE                     0.0000
DWPI_DETAILED_DESCRIPTIONS    73.0013
DWPI_NOVELTY                   0.0000
DWPI_USE                      56.0561
dtype: float64


In [262]:
# Count duplicates for each column
duplicate_counts = {col: features[col].duplicated().sum() for col in features.columns}

# Display results in terms of %
for col, count in duplicate_counts.items():
    print(f"Column '{col}' has {round(count/features.shape[0]*100,2)}% duplicate values.")

Column 'PATENT' has 0.0% duplicate values.
Column 'PUB_DATE' has 99.98% duplicate values.
Column 'COUNTRY' has 99.99% duplicate values.
Column 'PRIMARY_LANG' has 99.99% duplicate values.
Column 'IPC' has 23.05% duplicate values.
Column 'DWPI_TITLE' has 0.28% duplicate values.
Column 'DWPI_DETAILED_DESCRIPTIONS' has 75.06% duplicate values.
Column 'DWPI_NOVELTY' has 0.11% duplicate values.
Column 'DWPI_USE' has 57.11% duplicate values.


**It means PATENT, IPC, DWPI_TITLE, DWPI_NOVELTY are significanlty important to differentiate between patents and maybe for even classification purposes.**

**Whereas DWPI_DETAILED_DESCRIPTIONS and DWPI_USE with a lot of duplicate and missing values seem insignificant. Dropping them makes sense.**

**Columns "PUB_DATE", "COUNTRY" and "PRIMARY_LANG" also seem insignificant for differentiating between patents and their respective classes.**

I have to calculate the word embeddings for DWPI_TITLE and DWPI_NOVELTY columns. One clever way can be to combine the text in both columns to make it a single column and then calculate the word embeddings. The way we calculate embeddings for a sentence is once we have embedding for each word in the sentence, we simply add the embeddings for every single word to get embeddings for the sentence. This will be the most important feature to predict classes

In [263]:
# Dropping insignificant columns
features = features.drop(columns=['DWPI_DETAILED_DESCRIPTIONS', 'DWPI_USE'])

In [264]:
class_labels.nunique()

PATENT    344328
CLASS        287
dtype: int64

#### There are 287 unique classes for 344328 unique patents. But the class_labels DataFrame has 344328 unique patents with 461258 rows which means that some patents belong to multiple classes

In [265]:
class_labels = class_labels.groupby('PATENT')['CLASS'].apply(list).reset_index()
class_labels

,PATENT,CLASS
0,AR123668A1,[Q31]
1,AR123669A1,"[T01, P14]"
2,AR123694A1,[Q49]
3,AR123695A1,"[S03, J04]"
4,AR123703A1,[A93]
...,...,...
344323,ZA202213920B_,[P14]
344324,ZA202300109B_,[Q43]
344325,ZA202300118B_,"[S05, T01]"
344326,ZA202300175B_,"[A96, D22, F07, P32]"


In [266]:
class_labels.shape

(344328, 2)

In [267]:
unmatched_in_features = features[~features['PATENT'].isin(class_labels['PATENT'])]
unmatched_in_class_labels = class_labels[~class_labels['PATENT'].isin(features['PATENT'])]

print("Patents in features but not in class_labels:", len(unmatched_in_features))
print("Patents in class_labels but not in features:", len(unmatched_in_class_labels))

Patents in features but not in class_labels: 0
Patents in class_labels but not in features: 0


In [268]:
#### There are 344328 unique patents in both the data frames with exact match so doing a inner join
df = features.merge(class_labels, on='PATENT', how='inner')
df.shape

(344328, 8)

In [269]:
# Which columns have missing values after the join ?
missing_values_count = df.isna().sum()
print(missing_values_count)

PATENT           0
PUB_DATE         0
COUNTRY          0
PRIMARY_LANG     0
IPC             15
DWPI_TITLE       0
DWPI_NOVELTY     0
CLASS            0
dtype: int64


In [270]:
# Dropping rows where IPC values are missing. It wouldn't hurt because these 15 rows account for only 0.0044% of entire dataset.
df = df.dropna(subset=['IPC'])
missing_values_count = df.isna().sum()
print(missing_values_count)
# Finally, no missing values. The dataset is still big enough for models to be trained upon. 

PATENT          0
PUB_DATE        0
COUNTRY         0
PRIMARY_LANG    0
IPC             0
DWPI_TITLE      0
DWPI_NOVELTY    0
CLASS           0
dtype: int64


In [271]:
# Just looking at some examples where Class has multiple values
df_multi_class = df[df['CLASS'].apply(lambda x: len(x) > 1)]
df_multi_class.head()

,PATENT,PUB_DATE,COUNTRY,PRIMARY_LANG,IPC,DWPI_TITLE,DWPI_NOVELTY,CLASS
52,CN218439434U_,2023-02-03,CN,ZH,"['E21F000500', 'B03C0003017']",Automatic cleaning screen for tunnel air purif...,The utility model claims an automatic cleaning...,"[J01, X25]"
240,FR3124913A1,2023-01-06,FR,FR,"['H04L00410816', 'H04L001200', 'H04L004500', '...",Method for managing orchestration entity in so...,The method involves obtaining (E70) an indicat...,"[T01, W01]"
241,CN218227814U_,2023-01-06,CN,ZH,"['B29C004956', 'B29C004942', 'B29C004964', 'B2...",nan,The machine has a pull plate device whose bott...,"[A92, A32]"
245,WO2023279108A1,2023-01-05,WO,EN,"['C12N001586', 'A61K004800', 'C07K001447']",New nucleic acid molecule comprising an adeno-...,"Nucleic acid molecule, comprising an adeno-ass...","[B04, D16]"
247,CN115584655A_,2023-01-10,CN,ZH,"['C05B000700', 'D21B000116', 'D21C000104', 'D2...",Utilizing wheat straw crushed by-product bio-f...,Utilizing wheat straw crushed by-product bio-f...,"[F09, C04]"


In [273]:
# IPC is a list of codes but the list is parsed as string. Converting the string to an actual list
df['IPC'] = df['IPC'].apply(eval)
# Stripping the IPC codes to limit it to just the first 3 characters only
def mask_IPC(x):
    return list(set([ipc[:3] for ipc in x]))
df.IPC = df.IPC.apply(mask_IPC)

In [278]:
# We can drop, PATENT, PUB_DATE, PRIMARY_LANG and COUNTRY column. 
# We need correlations to come to that conclusion
df.head()

,PATENT,PUB_DATE,COUNTRY,PRIMARY_LANG,IPC,DWPI_TITLE,DWPI_NOVELTY,CLASS
0,CN218280954U_,2023-01-13,CN,ZH,[B01],Surface anti-sticking dust-removing filter bag...,The utility model claims a surface anti-sticki...,[A88]
1,CN218188237U_,2023-01-03,CN,ZH,[B01],Environment-friendly reusable cooling liquid f...,The utility model claims a new environment-fri...,[A88]
2,CN218442764U_,2023-02-03,CN,ZH,"[F24, B08]",Organic silicon modified epoxy heat conducting...,The utility model claims an organic silicon mo...,[A88]
3,CN218428090U_,2023-02-03,CN,ZH,[B25],"Electronic screen dismounting device, has lowe...",The utility model relates to screen disassembl...,[A88]
4,CN218195410U_,2023-01-03,CN,ZH,[B26],"Inclined cutting paper cutting device, has cut...",The utility model claims an inclined cutting p...,[A88]


In [281]:
# One-hot encoding IPC and CLASS columns
ipc_encoded = df.explode('IPC')['IPC'].str.get_dummies().groupby(level=0).sum()
class_encoded = df.explode('CLASS')['CLASS'].str.get_dummies().groupby(level=0).sum()

# Calculate overall correlation
overall_correlation = ipc_encoded.corrwith(class_encoded, axis=0).mean()
print("Overall Correlation between IPC and CLASS:", overall_correlation)

Overall Correlation between IPC and CLASS: -0.0002680179388479074


In [ ]:
# Make different dataframes for each class starting alphabet and plot alphabet count for each country
''' 
DWPI_TITLE and DWPI_NOVELTY columns can be combined into a single column and then embeddings can be generated for this combined column.
PATENT column doesnt seem to have any meaning because it is unique for each row. It seems like a primary key which is not important for classification.
Some patents have more than 1 Class/Alphabet associated with them such as [J01, X25]. 
The classifier at the top level needs to predict both J and X for this patent.
Then, I will send this patent to be further classified into the sections of J and X by using different classifiers trained for J and X respectively.
'''